# Script to assemble data from Bazzino and Roitman
This notebook will read in all files and save pickles with just stimulus associated signals

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tdt
import trompy as tp

import dill

In [2]:
DATAFOLDER = Path("..//data")
TANKFOLDER = Path("D://TestData//bazzino//to McCutcheon from Paula Bazzino")

# SubjectKey files give sex of rat

metadata = pd.read_csv(DATAFOLDER / "10NaCl_FileKey.csv")

In [3]:
# this function should get snips from the tank
# should add pre, post and other functions
def get_snips(tank):
    # preprocess streams
    
    data = tdt.read_block(tank)
    blue = data.streams["x65A"].data
    uv = data.streams["x05A"].data
    fs = data.streams["x05A"].fs
    
    filtered_sig = tp.processdata(blue, uv, fs=fs)
    
    # get epochs
    sol = data.epocs.sol_.onset
    
    #make snips
    snips = tp.snipper(filtered_sig, sol, fs=fs, pre=5, post=15, bins=200)[0]
    
    # baseline correction
    # and maybe scale them so they all go between -1 and 1 based on the same parameters
    
    # return snips
    return snips
    
# code to test get_snips function
# tank = TANKFOLDER / "PB23-220601-142444"
# data = get_snips(tank)
# plt.plot(data[0])

In [ ]:
# also make accompanying df that contains details of the rat and the condition and the time in session

def assemble_all_data(csv_path, tank_folder):
    metadata = pd.read_csv(csv_path)
    
    snips_array = []
    x_array = []
    
    for row in metadata.iterrows():
        tank = tank_folder / row[1]["Folder"]
        print(tank)
        try:
            snips_tmp = get_snips(tank)
            nsnips = len(snips_tmp)
            print(nsnips)
            snips_array.append(snips_tmp)
            
            x_array.append(pd.DataFrame(data={"trial": np.arange(nsnips),
                                              "id": row[1]["Subject"],
                                              "condition": row[1]["Physiological state"]
                                             }
                                       )
                          )           
        except:
            print("Error with tank for", row[1]["Subject"], row[1]['Physiological state'])
        
    return snips_array, x_array

snips_10NaCl, x_10NaCl = assemble_all_data(DATAFOLDER / "10NaCl_FileKey.csv", TANKFOLDER)
    

D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB23-220608-131619
read from t=0s to t=2614.16s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB24-220608-122638
read from t=0s to t=2643.15s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB33-220620-115414
read from t=0s to t=2616.75s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB39-220629-105034
read from t=0s to t=2679.85s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB391-220829-111452
read from t=0s to t=2683.06s
50
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB44-220829-121005
read from t=0s to t=2629.05s
50
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB61-221024-110730
read from t=0s to t=2626.98s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB62-221024-120029
read from t=0s to t=2606.66s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB64-221024-143444
read from t=0s to t=2637.56s
50
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB70-22102

c:\Users\jmc010\AppData\Local\mambaforge\envs\default\lib\site-packages\tdt\TDTbin2py.py:1437: Warning: sss/ buddy epoc not found, skipping
  warnings.warn(epocs.buddies[ii] + ' buddy epoc not found, skipping', Warning)


49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB61-221022-094209
read from t=0s to t=2651.47s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB62-221022-103215
read from t=0s to t=2638.67s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB64-221022-112221
read from t=0s to t=2673.06s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB70-221022-121226
read from t=0s to t=2531.26s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB39-220705-114000
read from t=0s to t=2660.96s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB391-220902-105101
read from t=0s to t=2604.58s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB44-220902-114732
read from t=0s to t=2630.1s
50
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB61-221027-091856
read from t=0s to t=2579.46s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB62-221027-100635
read from t=0s to t=2618.06s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB64-221

In [14]:
def tweak_x_array(x_array):

    df = pd.concat(x_array, axis=0)

    return (df
            .replace({"condition": {"Sodium Depleted": "deplete",
                                    "Sodium Replete": "replete",
                                    "Sodium Replete Experienced": "replete_exp",
                                    "Thirsty": "thirsty",                                    
                                    }})
            )

x_10NaCl = tweak_x_array(x_10NaCl)
snips_10NaCl = np.concatenate(snips_10NaCl, axis=0)

In [15]:
# assembles data for 45NaCl
snips_45NaCl, x_45NaCl = assemble_all_data(DATAFOLDER / "45NaCl_FileKey.csv", TANKFOLDER)
x_45NaCl = tweak_x_array(x_45NaCl)
snips_45NaCl = np.concatenate(snips_45NaCl, axis=0)


D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB26-220608-113244
read from t=0s to t=2605.16s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB27-220608-104154
read from t=0s to t=2626.46s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB30-220620-100529
read from t=0s to t=2625.06s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB31-220620-105832
read from t=0s to t=2685.05s
50
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB46-220926-133715
read from t=0s to t=2601.46s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB48-220926-121542
read from t=0s to t=2571.45s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB71-221123-113609
read from t=0s to t=2633.28s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB72-221123-122441
read from t=0s to t=2644.26s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB73-221123-131413
read from t=0s to t=2639.65s
49
D:\TestData\bazzino\to McCutcheon from Paula Bazzino\PB75-221123

In [16]:


data_to_save = {"snips_10NaCl": snips_10NaCl,
                "snips_45NaCl": snips_45NaCl,
                "x_10NaCl": x_10NaCl,
                "x_45NaCl": x_45NaCl
                }

with open(DATAFOLDER / "snips_data.pickle", "wb") as f:
    dill.dump(data_to_save, f)

In [17]:
snips_45NaCl.shape

(1375, 200)